In [274]:
import json
from nltk.stem import PorterStemmer
import urlextract
from nltk.corpus import stopwords
import re
from collections import Counter
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [54]:
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

In [253]:
tags = load_json("training\\tags.json")
types = load_json("training\\types.json")
events = load_json("data\\events.json")
places = load_json("data\\places.json")

In [254]:
class TextPreprocessor:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.url_extractor = urlextract.URLExtract()
        self.tag_regex = re.compile(r"<[^>]*>")
        self.email_regex = re.compile(r"[^\s]+@[^\s]+")
        self.number_regex = re.compile(r'\d+(?:\.\d*(?:[eE]\d+))?')
        self.dollar_regex = re.compile(r"[$]+")
        self.spaces_regex = re.compile(r"\s+")
        self.special_chars = [
            "<", "[", "^", ">", "+", "?", "!", "'", ".", ",", ":",
            "*", "%", "#", "_", "=", "-", "&", '/', '\\', '(', ')', ";", "\"", "«", "»", "|", "•", "—", "–", "●", "►", "\n"
        ]
        self.stop_words = set(stopwords.words('russian'))

    def preprocess_text(self, text):
        text = text.lower()
        text = self.remove_html_tags(text)
        text = self.replace_urls(text)
        text = self.replace_emails(text)
        text = self.replace_numbers(text)
        text = self.replace_dollar_signs(text)
        text = self.replace_places(text)
        text = self.stem_words(text)
        text = self.remove_special_characters(text)
        text = self.remove_stop_words(text)
        text = self.spaces_regex.sub(' ', text)
        return text.strip()

    def remove_html_tags(self, text):
        text = self.tag_regex.sub(" ", text).split(" ")
        text = filter(len, text)
        text = ' '.join(text)
        return text

    def replace_urls(self, text):
        urls = list(set(self.url_extractor.find_urls(text)))
        urls.sort(key=lambda u: len(u), reverse=True)
        for url in urls:
            text = text.replace(url, " httpaddr ")
        return text

    def replace_emails(self, text):
        return self.email_regex.sub(" emailaddr ", text)
    
    def replace_places(self, text):
        for place in places:
            text = text.replace(place, " place ")
        return text

    def replace_numbers(self, text):
        return self.number_regex.sub(" number ", text)

    def replace_dollar_signs(self, text):
        return self.dollar_regex.sub(" dollar ", text)

    def remove_special_characters(self, text):
        for char in self.special_chars:
            text = text.replace(str(char), " ")
        return text
    
    def remove_stop_words(self, text):
        for word in self.stop_words:
            text = text.replace(" %s " % word, " ")
        return text

    def stem_words(self, text):
        text = [self.stemmer.stem(token) for token in text.split(" ")]
        text = " ".join(text)
        return text
    
    
def convert_text_to_feature_vector(text):
    text = preprocessor.preprocess_text(text)
    words = text.split(' ')
    feature_vector = np.zeros(len(mapping))
    for word in words:
        if word in mapping:
            feature_vector[mapping[word] - 1] = 1
    return feature_vector

In [255]:
preprocessor = TextPreprocessor()

In [94]:
preprocessor.preprocess_text(tags[0][0])

'особенность формата конце вечера зрители выбирают самого смешного комика получает денежный приз зрителей вход свободный'

In [95]:
vocab = []
for text, tag in tags:
    text = preprocessor.preprocess_text(text)
    words = text.split(' ')
    word_counts = Counter(words)
    vocab.append(word_counts)

vocab = sum(vocab, Counter())
most_common = vocab.most_common(2000)
vocab = []
for (k, v) in most_common:
    vocab.append(k)

vocab = [(index + 1, word) for index, word in enumerate(sorted(vocab))]

In [96]:
mapping = {}
with open("training\\vocab", 'w+', encoding="utf-8") as f:
    for index, word in vocab:
        f.write("%s\t%s\n" % (index, word))
        mapping[word] = index

In [109]:
tag_data = [t[1] for t in tags]
tag_data = sorted(list(set([t.lower() for tt in tag_data for t in tt])))
tag_mapping = {}
for i, t in enumerate(tag_data):
    tag_mapping[t] = i

def convert_tags_to_vector(tag_list):
    v = np.zeros(len(tag_mapping))
    for t in tag_list:
        t = t.lower()
        v[tag_mapping[t]] = 1
    return v

In [176]:
type_data = [t[1] for t in types]
type_data = sorted(list(set([t.lower() for tt in type_data for t in tt])))
type_mapping = {}
for i, t in enumerate(type_data):
    type_mapping[t] = i

def convert_type_to_vector(type_value):
    v = np.zeros(len(type_mapping))
    for t in type_value:
        t = t.lower()
        v[type_mapping[t]] = 1
    return v

In [130]:
x = []
y = []

for text, tag in tags:
    feature_vector = convert_text_to_feature_vector(text, mapping)
    tag_vector = convert_tags_to_vector(tag, tag_mapping)
    x.append(feature_vector)
    y.append(tag_vector)
    
x = np.array(x)
y = np.array(y)

In [166]:
classifiers = []
for tag, tag_id in tag_mapping.items(): 
    print("Tag '%s'" % tag)
    y_train = []
    for v in y:
        y_train.append(v[tag_id])
    y_train = np.array(y_train)
    clf = svm.SVC(kernel='linear')
    clf.fit(x, y_train)
    print(clf.score(x, y_train))
    classifiers.append(clf)

Tag 'dj'
0.9980314960629921
Tag 'handmade'
1.0
Tag 'open air'
1.0
Tag 'авторская песня'
1.0
Tag 'аксессуары'
1.0
Tag 'активный отдых'
1.0
Tag 'акустика'
1.0
Tag 'альтернатива'
1.0
Tag 'балет'
1.0
Tag 'бардовская песня'
1.0
Tag 'баян'
1.0
Tag 'бег'
1.0
Tag 'бесплатно'
0.9990157480314961
Tag 'биатлон'
1.0
Tag 'блюз'
1.0
Tag 'боевые искусства'
1.0
Tag 'велосипед'
1.0
Tag 'водевиль'
1.0
Tag 'вокал'
1.0
Tag 'волонтерство'
1.0
Tag 'гастрономия'
1.0
Tag 'гонка'
1.0
Tag 'гранж'
1.0
Tag 'графика'
1.0
Tag 'детский'
1.0
Tag 'джаз'
1.0
Tag 'дизайн'
1.0
Tag 'для свиданий'
1.0
Tag 'драма'
1.0
Tag 'живопись'
1.0
Tag 'животные'
1.0
Tag 'игрушки'
1.0
Tag 'инди-поп'
1.0
Tag 'инди-рок'
1.0
Tag 'интерактивная'
1.0
Tag 'интерьер'
1.0
Tag 'искусство'
1.0
Tag 'история'
1.0
Tag 'йога'
1.0
Tag 'кавер-бэнд'
1.0
Tag 'кантри'
1.0
Tag 'кикер'
1.0
Tag 'классика'
1.0
Tag 'книги'
1.0
Tag 'комедия'
1.0
Tag 'кукольный'
1.0
Tag 'кулинария'
1.0
Tag 'лаунж'
1.0
Tag 'литературно-документальный'
1.0
Tag 'личность'
1.0
Tag '

In [167]:
def detect_tags(text):
    text = preprocessor.preprocess_text(text)
    vector = convert_text_to_feature_vector(text, mapping)
    result = []
    for i, clf in enumerate(classifiers):
        prediction = clf.predict([vector])
        if prediction[0] == 1:
            result.append(tag_data[i])
    return result

In [168]:
actual = 0
predicted = 0
for text, t in tags:
    p = set(detect_tags(text))
    t = set(v.lower() for v in t)
    actual += len(t)
    predicted += len(p & t)

print(actual)
print(predicted)
print(predicted / actual)

1327
1319
0.9939713639788997


In [180]:
x = []
y = []

for text, type_value in types:
    feature_vector = convert_text_to_feature_vector(text)
    type_vector = convert_type_to_vector(type_value)
    x.append(feature_vector)
    y.append(type_vector)
    
x = np.array(x)
y = np.array(y)

In [181]:
type_classifiers = []
for type_value, type_id in type_mapping.items(): 
    print("Type '%s'" % type_value)
    y_train = []
    for v in y:
        y_train.append(v[type_id])
    y_train = np.array(y_train)
    clf = svm.SVC(kernel='linear')
    clf.fit(x, y_train)
    print(clf.score(x, y_train))
    type_classifiers.append(clf)

Type 'автограф-сессия'
1.0
Type 'вечеринка'
0.9985590778097982
Type 'воркшоп'
1.0
Type 'выставка'
1.0
Type 'интенсив'
1.0
Type 'квест'
1.0
Type 'квиз'
1.0
Type 'кино'
1.0
Type 'конкурс'
1.0
Type 'конференция'
1.0
Type 'концерт'
0.989193083573487
Type 'лекция'
1.0
Type 'литературный вечер'
1.0
Type 'мастер-класс'
1.0
Type 'митап'
1.0
Type 'модный показ'
1.0
Type 'праздник'
1.0
Type 'презентация'
1.0
Type 'прием заявок'
1.0
Type 'семинар'
1.0
Type 'спектакль'
0.9920749279538905
Type 'творческая встреча'
1.0
Type 'тренинг'
1.0
Type 'турнир'
1.0
Type 'фестиваль'
1.0
Type 'форум'
1.0
Type 'хакатон'
1.0
Type 'шоу'
1.0
Type 'экскурсия'
1.0
Type 'ярмарка'
1.0


In [184]:
def detect_types(text):
    text = preprocessor.preprocess_text(text)
    vector = convert_text_to_feature_vector(text)
    result = []
    for i, clf in enumerate(type_classifiers):
        prediction = clf.predict([vector])
        if prediction[0] == 1:
            result.append(type_data[i])
    return result

In [185]:
actual = 0
predicted = 0
for text, t in types:
    p = set(detect_types(text))
    t = set(v.lower() for v in t)
    actual += len(t)
    predicted += len(p & t)

print(actual)
print(predicted)
print(predicted / actual)

1460
1446
0.9904109589041096


In [186]:
from telethon import TelegramClient, events, sync

In [203]:
api_id = 747366
api_hash = '9810a745cd9de2ac266e15d6152d99f4'

client = TelegramClient('session_name_2', api_id, api_hash)
await client.start()

Please enter your phone (or bot token): +375445822205
Please enter the code you received: 52883
Signed in successfully as Родион


In [205]:
async for i in client.iter_dialogs():
    print(i.title)

Telegram
Come here, Minsk
Берг
Родион
Vlad Birukov
Вероничка
Тананыч
KSiS OBOI 85864(1/2) (only students)
Ярмола
aaaaaaaaaaa
Dmitry Chabanenko


In [215]:
channel = await client.get_entity('Come here, Minsk')

In [259]:
events = list(set(open('data/events', encoding="utf-8").readlines()))
not_events = list(set(open('data/not_events', encoding="utf-8").readlines()))

In [260]:
event_vectors = [convert_text_to_feature_vector(event) for event in events]
not_event_vectors = [convert_text_to_feature_vector(not_event) for not_event in not_events]

In [280]:
y_train_events = np.zeros(len(event_vectors))
y_train_events.fill(1)
y_train_not_events = np.zeros(len(not_event_vectors))

x_train = np.concatenate((event_vectors, not_event_vectors))
y_train = np.concatenate((y_train_events, y_train_not_events))

In [281]:
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 0.5, 1, 1.5]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(x_train, y_train)
#    print("Training Accuracy:", (clf.score(x_train, y_train)) * 100, "%")
#    print("For c=", c, " Testing Accuracy:", (clf.score(x_test, y_test)) * 100, "%")

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.5, 1, 1.5], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [286]:
classifier = clf.best_estimator_

In [290]:
text = "тематика выставки мебель жилых зон загородных пространств офисов сектора horeca мебель премиум класса мебель предметы интерьера натурального дерева фурнитура материалы комплектующие инновации технологические решения дызайнпрастор дизайн интерьера декор пространства современные отделочные материалы конструкции дизайнерские решения жилых офисных общественных помещений рамках деловой программы участники проанонсируют новинки расскажут успешных кейсах поделятся опытом обсудят специфику ведения бизнеса современных условиях компаний работающих мебельном рынке будут организованы семинары маркетингу эффективных каналах рекламы продажам посетителей ждет ежегодный весенний дизайн amp декор форум площадка коммуникаций дизайнеров интерьера декораторов бесплатные дизайн консультации оформлению офисов кафе баров ресторанов также жилых домов квартир подведение итогов ряда конкурсов вход бесплатный условии регистрации регистрации number рублей режим работы number number number number"

In [300]:
not_text = "поздравляйте любимых оригинально в этом всегда поможет театр эстрады - поздравьте свою вторую половинку прямо со сцены театра! день всех влюблённых приглашаем провести, окунувшись в снежную историю любви для зрителей приготовили не просто концерт, а ещё и прекрасную возможность поздравить своих любимых с праздником прямо со сцены, а может признаться в любви или даже сделать предложение. в этот день все ваши желания и даже самые авантюрные поступки реальны! хотите поздравить свою вторую половинку? - заранее высылайте ваши поздравления/признания или другие интересные предложения для поздравления своих любимых на почту театра httpaddr с пометкой « number февраля», и они прозвучат со сцены в день концерта. не забудьте указать дату посещения. поторопитесь, ведь праздник не за горами! и не сомневайтесь, театр эстрады подготовил настоящий подарок в этот праздничный день всем влюблённым! а может, именно здесь вы встретите свою любовь"

In [301]:
vector = convert_text_to_feature_vector(not_text)

In [302]:
classifier.predict([vector])

array([0.])

In [307]:
for text in not_events:
    vector = convert_text_to_feature_vector(text)
    prediction = classifier.predict([vector])
    print(prediction[0] == 1.0)

False
False
True
False
False
False
False
True
False
False
False
False
False
False
True
True
True
True
False
False
False
True
False
False
True
False
True
False
False
False
True
True
False
True
False
False
True
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
True
False
False
False
True
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
True
False
False
False
False
True
True
False
True
True
False
False
True
False
False
False
False
False
False
False
Fals

In [309]:
with open("data\\not_events", "r", encoding="utf-8") as f:
    not_events = f.readlines()

with open("data\\not_events_2", "w+", encoding="utf-8") as f:
    for e in not_events:
        if len(e.strip()) > 2:
            f.write(e)
            
